## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('/home/itingzon.unicef/giga')
from src import sat_download
from utils import data_utils
from utils import config_utils
from utils import pred_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [2]:
iso_code="KEN"
adm_level="ADM2"
sum_threshold=5
buffer_size=150
spacing=150

config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AF.yaml")
sat_config_file = os.path.join(os.getcwd(), "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(os.getcwd(), "configs/sat_configs/issa_sat_creds.yaml")

config = config_utils.load_config(config_file)
sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [ ]:
tiles = pred_utils.batch_process_tiles(config, iso_code, spacing, buffer_size, sum_threshold, adm_level)

## Download Satellite Images

In [5]:
pred_utils.batch_download_sat_images(
    sat_config, sat_creds, config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)

INFO:root:Data dimensions: (26943, 7), CRS: EPSG:3857


Kitui Rural 0/290 total tiles: (26943, 5)


INFO:root:Data dimensions: (20012, 7), CRS: EPSG:3857


Kitui Central 1/290 total tiles: (20012, 5)


INFO:root:Data dimensions: (25382, 7), CRS: EPSG:3857


...

100%|████████████████████| 8598/8598 [2:18:06<00:00,  1.04it/s]                                                                                                             
INFO:root:Data dimensions: (18776, 7), CRS: EPSG:3857


Mavoko 9/290 total tiles: (18776, 5)


100%|████████████████████| 18776/18776 [5:02:55<00:00,  1.03it/s]                                                                                                           
INFO:root:Data dimensions: (20912, 7), CRS: EPSG:3857


Machakos Town 10/290 total tiles: (20912, 5)


 65%|████████████▉       | 13541/20912 [3:46:00<2:03:01,  1.00s/it]                                                                                                         

KeyboardInterrupt



## Run Model

In [ ]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.cnn_predict(tiles, iso_code, shapename, model_config, sat_dir, n_classes=2)